In [7]:
import cv2
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

written_text = ""
current_letter = ""

frame_counter = 0
frame_threshold = 30  

def recognize_gesture(landmarks):

    thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP].y
    index_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
    middle_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y

    if thumb_tip < index_tip:
        return "write"
    
    if index_tip < middle_tip:
        return "next"

    return None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    frame = cv2.flip(frame, 1)

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            gesture = recognize_gesture(hand_landmarks.landmark)

            if gesture == "write":
                frame_counter += 1
                if frame_counter >= frame_threshold:
                    if current_letter == "":
                        current_letter = "A"
                    else:
                        current_letter = chr((ord(current_letter) - 65 + 1) % 26 + 65)
                    frame_counter = 0

            elif gesture == "next":
                if current_letter:
                    written_text += current_letter
                    current_letter = ""
                    frame_counter = 0

    cv2.putText(frame, f"Text: {written_text}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f"Current: {current_letter}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('MediaPipe Hands - Alphabet Writer', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
